 ### Text Classification

In [1]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.datasets import load_files


import autokeras as ak

2023-01-30 16:07:15.011684: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-30 16:07:16.438491: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:
2023-01-30 16:07:16.438674: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:
2023-01-30 16:07:16.438689: W tensorflow/compiler/tf2tensorrt/util

In [2]:
#Prepare Dataset
dataset = tf.keras.utils.get_file(
    fname = "aclImdb.tar.gz",
    origin = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
    extract = True)

In [3]:
IMDB_DATADIR = os.path.join(os.path.dirname(dataset), "aclImdb")

In [4]:
print(IMDB_DATADIR)

/home/dudco0040/.keras/datasets/aclImdb


In [5]:
classes = ["pos","neg"]
train_data = load_files(os.path.join(IMDB_DATADIR,"train"), shuffle = True, categories= classes)
test_data = load_files(os.path.join(IMDB_DATADIR,"test"), shuffle = False, categories= classes)

x_train = np.array(train_data.data)
y_train = np.array(train_data.target)
x_test = np.array(test_data.data)
y_test = np.array(test_data.target)


print(x_train.shape)
print(y_train.shape)
#print(x_train)
print(y_train)
print(x_train[0][:50])

(25000,)
(25000,)
[1 0 1 ... 0 0 0]
b'Zero Day leads you to think, even re-think why two'


In [6]:
# Text Classification
clf = ak.TextClassifier(overwrite = True, max_trials = 1)
clf.fit(x_train, y_train, epochs = 2)

Trial 1 Complete [00h 01m 54s]
val_loss: 0.2820516526699066

Best val_loss So Far: 0.2820516526699066
Total elapsed time: 00h 01m 54s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
782/782 [==============================] - 83s 103ms/step - loss: 0.4327 - accuracy: 0.7800
Epoch 2/2
782/782 [==============================] - 11s 14ms/step - loss: 0.2350 - accuracy: 0.9049


INFO:tensorflow:Assets written to: ./text_classifier/best_model/assets


INFO:tensorflow:Assets written to: ./text_classifier/best_model/assets


In [7]:
predicted_y = clf.predict(x_test)
print(clf.evaluate(x_test,y_test))

782/782 [==============================] - 8s 10ms/step - loss: 0.2705 - accuracy: 0.8916
[0.27049750089645386, 0.8915600180625916]


In [8]:
# Validation Data Split
clf.fit(x_train,y_train,
       vlidation_split = 0.15)  # option: vlidation_split

In [9]:
split = 5000
x_val = x_train[split:]
y_val = y_train[split:]
x_train = x_train[split:]
y_train = y_train[split:]
clf.fit(x_train, y_train, epoch = 2, validation_data = (x_val, y_val))

In [10]:
input_node = ak.TextInput()  # input node
output_node = ak.TextBlock(block_type = "ngram")(input_node)
output_node = ak.ClassificationHead()(output_node)   # output heads

# The user only specifies the input node and output heads.
# input node와 output heads를 지정해주면 나머지 부분은 추론을 통해 적용한다. 
clf = ak.AutoModel(inputs = input_node, outputs = output_node, overwrite = True, max_trials = 2)
#overwrite: 같은 이름의프로젝트가 있으면 프로젝트 덮어 씌우기 (defualt = False)
#max_trials: 시도할 수 있는 keras 모델의 최대 개수 (defualt = 100)
#max_model_size: 모델 매개변수의 최대 스칼라 수. 크기가 지정된 스칼라 수를 넘을 경우, 모델 거부
clf.fit(x_train, y_train, epochs = 2)

Trial 2 Complete [00h 00m 46s]
val_loss: 0.322414755821228

Best val_loss So Far: 0.322414755821228
Total elapsed time: 00h 01m 31s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/2
625/625 [==============================] - 12s 16ms/step - loss: 0.3964 - accuracy: 0.8412
Epoch 2/2
625/625 [==============================] - 10s 15ms/step - loss: 0.2553 - accuracy: 0.8988
INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


In [11]:
train_set = tf.data.Dataset.from_tensor_slices(((x_train,),(y_train,))).batch(32)
test_set = tf.data.Dataset.from_tensor_slices(((x_test,),(y_test,))).batch(32)

clf = ak.TextClassifier(overwrite = True, max_trials = 2)
clf.fit(train_set, epochs = 2)

pedicted_y = clf.predict(test_set)
print(clf.evaluate(test_set))

Trial 2 Complete [00h 01m 51s]
val_loss: 0.36551520228385925

Best val_loss So Far: 0.2861657738685608
Total elapsed time: 00h 03m 06s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/2
625/625 [==============================] - 65s 101ms/step - loss: 0.4659 - accuracy: 0.7539
Epoch 2/2
625/625 [==============================] - 9s 14ms/step - loss: 0.2528 - accuracy: 0.8965


INFO:tensorflow:Assets written to: ./text_classifier/best_model/assets


INFO:tensorflow:Assets written to: ./text_classifier/best_model/assets


782/782 [==============================] - 8s 10ms/step - loss: 0.2754 - accuracy: 0.8882
[0.2753950357437134, 0.888159990310669]


### Structured Data Classification

In [12]:
# Prepare Data
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [13]:
# StructuredDataClassifier 사용
clf = ak.StructuredDataClassifier(overwrite = True, max_trials = 5)   #It tries 5 different models.
clf.fit(train_file_path, "survived", epochs = 10)

Trial 5 Complete [00h 00m 08s]
val_accuracy: 0.843478262424469

Best val_accuracy So Far: 0.8782608509063721
Total elapsed time: 00h 00m 37s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/10
20/20 [==============================] - 2s 8ms/step - loss: 0.6586 - accuracy: 0.6220
Epoch 2/10
20/20 [==============================] - 0s 8ms/step - loss: 0.5734 - accuracy: 0.7671
Epoch 3/10
20/20 [==============================] - 0s 8ms/step - loss: 0.5149 - accuracy: 0.7943
Epoch 4/10
20/20 [==============================] - 0s 7ms/step - loss: 0.4750 - accuracy: 0.8070
Epoch 5/10
20/20 [==============================] - 0s 8ms/step - loss: 0.4491 - accuracy: 0.8214
Epoch 6/10
20/20 [==============================] - 0s 8ms/step - loss: 0.4333 - accuracy: 0.8166
Epoch 7/10
20/20 [==============================] - 0s 8ms/step - loss: 0.4232 - accuracy: 0.8166
Epoch 8/10
20/20 [==============================] - 0s 8ms/step - loss: 0.4160 - accuracy: 0.8198
Epoch 9/10
20/20 [==============================] - 0s 8ms/step - loss: 0.4104 - accuracy: 0.8230
Epoch 10/10
20/20 [==============================] - 0s 7ms/step - loss: 0.4060 - accuracy: 0.8309
INFO:tensorflow:Ass

INFO:tensorflow:Assets written to: ./structured_data_classifier/best_model/assets


In [14]:
# Predicted with the Best model.
predicted_y = clf.predict(test_file_path)
print(clf.evaluate(test_file_path, "survived"))

9/9 [==============================] - 0s 6ms/step - loss: 0.4389 - accuracy: 0.7879
[0.4389197826385498, 0.7878788113594055]


In [15]:
# The example above shows how to use the CSV files directly. 
x_train = pd.read_csv(train_file_path)
print(type(x_train))
y_train = x_train.pop("survived")
print(type(y_train))

y_train = pd.DataFrame(y_train)
print(type(y_train))

x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
print(x_train)
print(y_train)

x_test = pd.read_csv(test_file_path)
y_test = x_test.pop("survived")

clf = ak.StructuredDataClassifier(overwrite = True, max_trials = 3)
clf.fit(x_train, y_train, epochs = 10)
predicted_y = clf.predict(x_test)
print(clf.evaluate(x_test, y_test))

Trial 3 Complete [00h 00m 07s]
val_accuracy: 0.852173924446106

Best val_accuracy So Far: 0.852173924446106
Total elapsed time: 00h 00m 21s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/10
20/20 [==============================] - 2s 10ms/step - loss: 0.6366 - accuracy: 0.6874
Epoch 2/10
20/20 [==============================] - 0s 8ms/step - loss: 0.5582 - accuracy: 0.7496
Epoch 3/10
20/20 [==============================] - 0s 8ms/step - loss: 0.5071 - accuracy: 0.7959
Epoch 4/10
20/20 [==============================] - 0s 7ms/step - loss: 0.4747 - accuracy: 0.8038
Epoch 5/10
20/20 [==============================] - 0s 8ms/step - loss: 0.4531 - accuracy: 0.8086
Epoch 6/10
20/20 [==============================] - 0s 7ms/step - loss: 0.4376 - accuracy: 0.8102
Epoch 7/10
20/20 [==============================] - 0s 8ms/step - loss: 0.4260 - accuracy: 0.8150
Epoch 8/10
20/20 [==============================] - 0s 8ms/step - loss: 0.4173 - accuracy: 0.8182
Epoch 9/10
20/20 [==============================] - 0s 8ms/step - loss: 0.4108 - accuracy: 0.8262
Epoch 10/10
20/20 [==============================] - 0s 8ms/step - loss: 0.4055 - accuracy: 0.8325
INFO:tensorflow:As

INFO:tensorflow:Assets written to: ./structured_data_classifier/best_model/assets


9/9 [==============================] - 0s 6ms/step - loss: 0.4426 - accuracy: 0.7803
[0.4425978362560272, 0.7803030014038086]


In [16]:
train_set = tf.data.Dataset.from_tensor_slices((x_train.astype(str), y_train))
test_set = tf.data.Dataset.from_tensor_slices((x_test.to_numpy().astype(str), y_test))
print(train_set)
print(test_set)

<TensorSliceDataset element_spec=(TensorSpec(shape=(9,), dtype=tf.string, name=None), TensorSpec(shape=(1,), dtype=tf.int64, name=None))>
<TensorSliceDataset element_spec=(TensorSpec(shape=(9,), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>


In [17]:
clf = ak.StructuredDataClassifier(overwrite = True, max_trials = 3)
clf.fit(train_set, epochs = 10)
predicted_y = clf.predict(test_set)
print(clf.evaluate(test_set))

Trial 3 Complete [00h 00m 08s]
val_accuracy: 0.8695651888847351

Best val_accuracy So Far: 0.8782608509063721
Total elapsed time: 00h 00m 25s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/10
20/20 [==============================] - 2s 10ms/step - loss: 0.6667 - accuracy: 0.6380
Epoch 2/10
20/20 [==============================] - 0s 8ms/step - loss: 0.6028 - accuracy: 0.6970
Epoch 3/10
20/20 [==============================] - 0s 8ms/step - loss: 0.5460 - accuracy: 0.7528
Epoch 4/10
20/20 [==============================] - 0s 8ms/step - loss: 0.5145 - accuracy: 0.7799
Epoch 5/10
20/20 [==============================] - 0s 8ms/step - loss: 0.4916 - accuracy: 0.7927
Epoch 6/10
20/20 [==============================] - 0s 9ms/step - loss: 0.4716 - accuracy: 0.8102
Epoch 7/10
20/20 [==============================] - 0s 9ms/step - loss: 0.4610 - accuracy: 0.8182
Epoch 8/10
20/20 [==============================] - 0s 9ms/step - loss: 0.4488 - accuracy: 0.8102
Epoch 9/10
20/20 [==============================] - 0s 8ms/step - loss: 0.4359 - accuracy: 0.8230
Epoch 10/10
20/20 [==============================] - 0s 7ms/step - loss: 0.4205 - accuracy: 0.8309
INFO:tensorflow:As

INFO:tensorflow:Assets written to: ./structured_data_classifier/best_model/assets


9/9 [==============================] - 0s 5ms/step - loss: 0.4435 - accuracy: 0.7917
[0.44353485107421875, 0.7916666865348816]


In [18]:
clf = ak.StructuredDataClassifier(column_names = ["sex","age","n_siblings_spoueses","parch",
                                                  "fare","class","deck","embark_town","alone"],
                                  column_types = {"sex": "categorical", "fare": "numerical"},
                                  max_trials = 10, 
                                  overwrite = True)
# 컬럼명과 데이터 타입을 지정할 수 있음
# type이 지정되지 않은 경우, train dataset에서 추론하여 지정.

clf.fit(train_set, epochs = 10)
predicted_y = clf.predict(test_set)
print(clf.evaluate(test_set))

Trial 10 Complete [00h 00m 09s]
val_accuracy: 0.8782608509063721

Best val_accuracy So Far: 0.8782608509063721
Total elapsed time: 00h 01m 31s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/10
20/20 [==============================] - 3s 9ms/step - loss: 0.6901 - accuracy: 0.5774
Epoch 2/10
20/20 [==============================] - 0s 9ms/step - loss: 0.6088 - accuracy: 0.7751
Epoch 3/10
20/20 [==============================] - 0s 9ms/step - loss: 0.5317 - accuracy: 0.8230
Epoch 4/10
20/20 [==============================] - 0s 8ms/step - loss: 0.4684 - accuracy: 0.8198
Epoch 5/10
20/20 [==============================] - 0s 9ms/step - loss: 0.4376 - accuracy: 0.8166
Epoch 6/10
20/20 [==============================] - 0s 8ms/step - loss: 0.4230 - accuracy: 0.8325
Epoch 7/10
20/20 [==============================] - 0s 8ms/step - loss: 0.4137 - accuracy: 0.8357
Epoch 8/10
20/20 [==============================] - 0s 8ms/step - loss: 0.4067 - accuracy: 0.8357
Epoch 9/10
20/20 [==============================] - 0s 8ms/step - loss: 0.4012 - accuracy: 0.8373
Epoch 10/10
20/20 [==============================] - 0s 8ms/step - loss: 0.3969 - accuracy: 0.8405
INFO:tensorflow:Ass

INFO:tensorflow:Assets written to: ./structured_data_classifier/best_model/assets


9/9 [==============================] - 0s 6ms/step - loss: 0.4400 - accuracy: 0.8106
[0.43996670842170715, 0.810606062412262]


In [19]:
# Layer
input_node = ak.StructuredDataInput()
output_node = ak.StructuredDataBlock(categorical_encoding = True)(input_node)
output_node = ak.RegressionHead()(output_node)

reg = ak.AutoModel(inputs = input_node, outputs = output_node, overwrite = True, max_trials = 3)
reg.fit(x_train, y_train, epochs = 10)

Trial 3 Complete [00h 00m 07s]
val_loss: 0.14227251708507538

Best val_loss So Far: 0.14227251708507538
Total elapsed time: 00h 00m 22s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/10
20/20 [==============================] - 2s 11ms/step - loss: 0.7124 - mean_squared_error: 0.7124
Epoch 2/10
20/20 [==============================] - 0s 9ms/step - loss: 0.5229 - mean_squared_error: 0.5229
Epoch 3/10
20/20 [==============================] - 0s 9ms/step - loss: 0.4421 - mean_squared_error: 0.4421
Epoch 4/10
20/20 [==============================] - 0s 8ms/step - loss: 0.4393 - mean_squared_error: 0.4393
Epoch 5/10
20/20 [==============================] - 0s 8ms/step - loss: 0.3977 - mean_squared_error: 0.3977
Epoch 6/10
20/20 [==============================] - 0s 8ms/step - loss: 0.3556 - mean_squared_error: 0.3556
Epoch 7/10
20/20 [==============================] - 0s 10ms/step - loss: 0.3625 - mean_squared_error: 0.3625
Epoch 8/10
20/20 [==============================] - 0s 9ms/step - loss: 0.3418 - mean_squared_error: 0.3418
Epoch 9/10
20/20 [==============================] - 0s 8ms/step - loss: 0.3000 - mean_squared_error: 0.3000
Epoch 10/10
20/20 [=======

INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


In [20]:
# You can even also use more fine grained blocks to customize the search space even further. 
input_node = ak.StructuredDataInput()
output_node = ak.CategoricalToNumerical()(input_node)
output_node = ak.DenseBlock()(output_node)
output_node = ak.RegressionHead()(output_node)

reg = ak.AutoModel(inputs = input_node, outputs = output_node, max_trials = 3, overwrite = True)
reg.fit(x_train, y_train, epochs = 10)

Trial 3 Complete [00h 00m 13s]
val_loss: 0.18673808872699738

Best val_loss So Far: 0.1735192835330963
Total elapsed time: 00h 00m 28s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/10
20/20 [==============================] - 3s 12ms/step - loss: 0.8318 - mean_squared_error: 0.8318
Epoch 2/10
20/20 [==============================] - 0s 10ms/step - loss: 0.3042 - mean_squared_error: 0.3042
Epoch 3/10
20/20 [==============================] - 0s 10ms/step - loss: 0.2473 - mean_squared_error: 0.2473
Epoch 4/10
20/20 [==============================] - 0s 10ms/step - loss: 0.2197 - mean_squared_error: 0.2197
Epoch 5/10
20/20 [==============================] - 0s 9ms/step - loss: 0.2053 - mean_squared_error: 0.2053
Epoch 6/10
20/20 [==============================] - 0s 10ms/step - loss: 0.1939 - mean_squared_error: 0.1939
Epoch 7/10
20/20 [==============================] - 0s 10ms/step - loss: 0.1854 - mean_squared_error: 0.1854
Epoch 8/10
20/20 [==============================] - 0s 10ms/step - loss: 0.1786 - mean_squared_error: 0.1786
Epoch 9/10
20/20 [==============================] - 0s 10ms/step - loss: 0.1731 - mean_squared_error: 0.1731
Epoch 10/10
20/20 [=

INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


In [21]:
model = reg.export_model()  # export best model. 
model.summary()
#model.predict(x_train)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9)]               0         
                                                                 
 multi_category_encoding (Mu  (None, 9)                0         
 ltiCategoryEncoding)                                            
                                                                 
 dense (Dense)               (None, 32)                320       
                                                                 
 batch_normalization (BatchN  (None, 32)               128       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056  